In [0]:
%pip install -U langchain langgraph databricks-sdk databricks-vectorsearch
%pip install databricks-sdk[openai]
%pip install -U databricks-agents>=0.16.0 mlflow>=2.20.2 databricks-langchain databricks-openai
dbutils.library.restartPython()

In [0]:
import mlflow
from datetime import datetime
import json 

disable_notice=True

CATALOG = "workspace"
SCHEMA = "genai_demo"

## Flow-based PM Agent

In [0]:
# load the registered agent
FLOW_AGENT_NAME = 'flow_pm_agent'
FLOW_AGENT_VERSION = 1
FLOW_AGENT_NAME_FULL = f"models:/{CATALOG}.{SCHEMA}.{FLOW_AGENT_NAME}/{FLOW_AGENT_VERSION}"
flow_agent = mlflow.pyfunc.load_model(FLOW_AGENT_NAME_FULL)

Machine's normal status:
- Temperature: 20 ~ 35
- Vibration: 1.0 ~ 2.0
- Pressure: 30 ~ 35

### Normal Sensor Data

In [0]:
# Normal Sensor Data
sensor_data_normal = {
    "timestamp": str(datetime.now()), 
    "machine_id": 1, 
    "temperature": 20, 
    "vibration": 1.5, 
    "pressure": 30
    }

request = {"messages": [{"role": "user", "content": json.dumps(sensor_data_normal)}]}
response = flow_agent.predict(request)
print(response['messages'][0]['content'])


### Anomalous Sensor Data

In [0]:
# Anomaly Sensor Data
sensor_data_anomaly = {
    "timestamp": str(datetime.now()), 
    "machine_id": 1, 
    "temperature": 45, 
    "vibration": 3.5, 
    "pressure": 25
    }

request = {"messages": [{"role": "user", "content": json.dumps(sensor_data_anomaly)}]}
response = flow_agent.predict(request)
print(response['messages'][0]['content'])

## Conversational PM Agent

In [0]:
# load the registered agent
CHAT_AGENT_NAME = 'chat_pm_agent'
CHAT_AGENT_VERSION = 1
CHAT_AGENT_NAME_FULL = f"models:/{CATALOG}.{SCHEMA}.{CHAT_AGENT_NAME}/{CHAT_AGENT_VERSION}"
chat_agent = mlflow.pyfunc.load_model(CHAT_AGENT_NAME_FULL)

### 1. General Query

In [0]:
query = "Hello, what can you assist?"
response = chat_agent.predict({"messages":[{"role": "user", "content": query}]})
print(response['messages'][-1]['content'])

### 2. Free-form Maintenance Query 

In [0]:
query = "The machine seems to be running hot. Could you look through the manuals to identify possible causes and suggest a solution?"
response = chat_agent.predict({"messages":[{"role": "user", "content": query}]})
print(response['messages'][-1]['content'])

### 3. Query Containing Sensor Data (Normal)

In [0]:
query = "The sensor readings from another machine are: temperature at 20°C, vibration at 1.5mm/s, and pressure at 30 PSI. Is it functioning as expected?"
response = chat_agent.predict({"messages":[{"role": "user", "content": query}]})
print(response['messages'][-1]['content'])

### 4. Query with Sensor Data (Anomalous)

In [0]:
query = "The most recent sensor data from a different machine shows a temperature of 45°C, a vibration level of 3.5mm/s, and a pressure reading of 25 PSI. What do you think?"
response = chat_agent.predict({"messages":[{"role": "user", "content": query}]})
print(response['messages'][-1]['content'])

### 5. Follow-up Request for RCA and Resolution Suggestion

In [0]:
query = "Yes, proceed with a root cause analysis (RCA) and recommending appropriate resolutions for the issue"
response = chat_agent.predict({"messages":[{"role": "user", "content": query}]})
print(response['messages'][-1]['content'])

### All History

In [0]:
# Print the entire chat history
for message in response['messages']:
    role = message['role']
    content = message['content']
    print(f"=================================={role}==================================")
    print(content)